In [5]:
from tabulate import tabulate
from reservoirpy.nodes import (Reservoir, IPReservoir, NVAR, LMS, RLS, Ridge, ESN, Input)
from reservoirpy.datasets import (lorenz, mackey_glass, narma)
from reservoirpy.observables import (rmse, rsquare, nrmse, mse)
import numpy as np
import copy
import math
import matplotlib.pyplot as plt
from numpy import linalg as LA

Following the same setup as "Growing Echo-State Network With Multiple Subreservoirs"

In [53]:
# from reservoirpy.utils.random import rand_generator
# from reservoirpy.utils.validation import check_vector
# from reservoirpy.datasets import get_seed

# def narma(
#     n_timesteps: int,
#     order: int = 30,
#     a1: float = 0.2,
#     a2: float = 0.04,
#     b: float = 1.5,
#     c: float = 0.001,
#     x0 = [0.0],
#     seed = None,
# ) -> np.ndarray:
#     if seed is None:
#         seed = get_seed()
#     rs = rand_generator(seed)

#     y = np.zeros((n_timesteps + order, 1))

#     x0 = check_vector(np.atleast_2d(np.asarray(x0)))
#     y[: x0.shape[0], :] = x0

#     noise = rs.uniform(0, 1, size=(n_timesteps + order, 1))
#     for t in range(order, n_timesteps + order - 1):
#         y[t + 1] = (
#             a1 * y[t]
#             + a2 * y[t] * np.sum(y[t - order : t])
#             + b * noise[t - order] * noise[t]
#             + c
#         )
#     return y[order:, :]

In [13]:
def getData():
    data = np.array(narma(n_timesteps=10000, order=10, a1=0.3, a2=0.05, b=1.5, c=0.1))
    train = data[:2000]
    val = data[2000:3000]
    test = data[3000:]
    trainX = train[:-10]
    trainY = train[10:]
    valX = val[:-10]
    valY = val[10:]
    testX = test[:-10]
    testY = test[10:]
    return trainX, trainY, valX, valY, testX, testY

def nrmseGESN(yTrue, preds):
    norm = 0
    mse = 0
    targetMean = np.mean(yTrue)
    for i, target in enumerate(yTrue):
        norm+=(target - targetMean)**2
        mse+=(target - preds[i])**2
    return math.sqrt(mse/norm)
    # return nrmse(yTrue, preds, norm_value=norm)



# def temp():
#     # reservoir1 = NVAR(delay=2, order=2, strides=2)
#     # reservoir2 = NVAR(delay=2, order=2, strides=1)
#     # input2 = Input(input_dim=5)
#     # readout = Ridge(output_dim=1, ridge=1e-5)
#     # esn = reservoir1 >> input2 >> reservoir2 >> readout & reservoir1 >> readout
#     esn = (NVAR(delay=3, order=3, strides=2) >> NVAR(delay=3, order=3, strides=2)) >> Ridge(output_dim=1, ridge=1e-5)
#     trainX, trainY, testX, testY = getData()

#     esn.fit(trainX, trainY, warmup=1000)

#     esn2 = (NVAR(delay=3, order=3, strides=2) >> NVAR(delay=3, order=3, strides=2)) >> Ridge(output_dim=1, ridge=1e-5)
#     esnPreds = copy.deepcopy(esn).run(trainX)
#     esn2.fit(esnPreds, trainY, warmup=1000)

#     preds1 = esn.run(testX)
#     preds2 = esn2.run(preds1)
#     return nrmse(testY, preds2)

# abc = temp()
# print(abc)

In [21]:
def nmse(y, o):
    assert len(o) == len(y), "Both arrays must have the same length."
    sigma2 = np.var(y)  # variance of y
    i = len(y)
    error = sum((o[t] - y[t])**2 for t in range(i))
    return error / (i * sigma2)

def nrmse2(yTrue, preds):
    norm = len(yTrue) * np.var(yTrue)
    mse = 0
    targetMean = np.mean(yTrue)
    for i, target in enumerate(yTrue):
        mse+=(preds[i] - target)**2
    return math.sqrt(mse/norm)

def testDefault():
    esn = Reservoir(units=100, sr=0.95) >> Ridge(output_dim=1)
    trainX, trainY, testX, testY, _, _ = getData()

    esn.fit(trainX, trainY, warmup=100)
    preds = esn.run(testX)
    return nrmse2(testY, preds)

testDefault()

Running Model-12: 1990it [00:00, 22955.88it/s]         
Running Model-12: 100%|██████████| 1/1 [00:00<00:00, 11.26it/s]


Fitting node Ridge-12...


Running Model-12: 990it [00:00, 23361.21it/s]          


0.8932132848057498

In [52]:
def testDefault():
    esn = Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Ridge(output_dim=1, ridge=1e-5)
    trainX, trainY, testX, testY = getData()

    esn.fit(trainX, trainY, warmup=1000)
    preds = esn.run(testX)
    return nrmseGESN(testY, preds)

def testDeep():
    esn = (Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)) >> Ridge(output_dim=1, ridge=1e-5)
    trainX, trainY, testX, testY = getData()

    esn.fit(trainX, trainY, warmup=1000)
    preds = esn.run(testX)
    return nrmseGESN(testY, preds)

def testDeepCascade():
    esn = (Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)) >> Ridge(output_dim=1, ridge=1e-5)
    trainX, trainY, testX, testY = getData()

    esn.fit(trainX, trainY, warmup=1000)

    esn2 = (Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)) >> Ridge(output_dim=1, ridge=1e-5)
    esnPreds = copy.deepcopy(esn).run(trainX)
    esn2.fit(esnPreds, trainY, warmup=1000)

    preds1 = esn.run(testX)
    preds2 = esn2.run(preds1)
    return nrmseGESN(testY, preds2)

def testCascade():
    esn = Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Ridge(output_dim=1, ridge=1e-5)
    trainX, trainY, testX, testY = getData()

    esn.fit(trainX, trainY, warmup=1000)

    esn2 = Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Ridge(output_dim=1, ridge=1e-5)
    esnPreds = copy.deepcopy(esn).run(trainX)
    esn2.fit(esnPreds, trainY, warmup=1000)

    preds1 = esn.run(testX)
    preds2 = esn2.run(preds1)
    return nrmseGESN(testY, preds2)

def testErrorCascade():
    esn = Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Ridge(output_dim=1, ridge=1e-5)
    trainX, trainY, testX, testY = getData()

    esn.fit(trainX, trainY, warmup=1000)

    esn2 = Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Ridge(output_dim=1, ridge=1e-5)
    esnPreds = copy.deepcopy(esn).run(trainX)
    errors = esnPreds - trainY
    esn2.fit(esnPreds, errors, warmup=1000)

    preds1 = esn.run(testX)
    predictedErrors = esn2.run(preds1)
    preds = preds1 - predictedErrors
    return nrmseGESN(testY, preds)

def testDeepErrorCascade():
    esn = (Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)) >> Ridge(output_dim=1, ridge=1e-5)
    trainX, trainY, testX, testY = getData()

    esn.fit(trainX, trainY, warmup=1000)

    esn2 = (Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Reservoir(units=100, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)) >> Ridge(output_dim=1, ridge=1e-5)
    esnPreds = copy.deepcopy(esn).run(trainX)
    errors = esnPreds - trainY
    esn2.fit(esnPreds, errors, warmup=1000)

    preds1 = esn.run(testX)
    predictedErrors = esn2.run(preds1)
    preds = preds1 - predictedErrors
    return nrmseGESN(testY, preds)

def testNVAR():
    esn = NVAR(delay=8, order=3, strides=4) >> Ridge(output_dim=1, ridge=1e-5)
    trainX, trainY, testX, testY = getData()

    esn.fit(trainX, trainY, warmup=1000)
    preds = esn.run(testX)
    return nrmseGESN(testY, preds)

def testDeepCascadeNVAR():
    esn = (NVAR(delay=8, order=3, strides=4) >> Reservoir(units=100, sr=0.8, rc_connectivity=0.01)) >> Ridge(output_dim=1, ridge=1e-5)
    trainX, trainY, testX, testY = getData()

    esn.fit(trainX, trainY, warmup=1000)

    esn2 = (NVAR(delay=8, order=3, strides=4) >> Reservoir(units=100, sr=0.8, rc_connectivity=0.01)) >> Ridge(output_dim=1, ridge=1e-5)
    esnPreds = copy.deepcopy(esn).run(trainX)
    esn2.fit(esnPreds, trainY, warmup=1000)

    preds1 = esn.run(testX)
    preds2 = esn2.run(preds1)
    return nrmseGESN(testY, preds2)

error1 = testDefault()
error2 = testCascade()
error4 = testDeep()
error5 = testDeepCascade()
error6 = testDeepCascadeNVAR()
error7 = testErrorCascade()
error8 = testDeepErrorCascade()
error9 = testNVAR()
print("==================NARMA NRMSE==================")
print('Normal training error:', error1)
print('NVAR training error:', error9)
print('Cascading ESN error:', error2)
print('Deep ESN error:', error4)
print('Deep + cascade error:', error5)
print('Deep + cascade + NVAR error:', error6)
print('Error cascade error:', error7)
print('Deep error cascade error:', error8)

Running Model-108: 2799it [00:00, 20927.30it/s]?, ?it/s]
Running Model-108: 100%|██████████| 1/1 [00:00<00:00,  7.36it/s]


Fitting node Ridge-73...


Running Model-108: 2799it [00:00, 21778.06it/s]         
Running Model-109: 2799it [00:00, 19509.59it/s]?, ?it/s]
Running Model-109: 100%|██████████| 1/1 [00:00<00:00,  6.80it/s]


Fitting node Ridge-74...


Running Model-109: 2799it [00:00, 22042.71it/s]         
Running Model-110: 2799it [00:00, 20378.37it/s]?, ?it/s]
Running Model-110: 100%|██████████| 1/1 [00:00<00:00,  7.13it/s]


...

Fitting node Ridge-84...


Running Model-124: 2799it [00:00, 13051.70it/s]         
Running Model-126: 2799it [00:00, 13211.31it/s]         
Running Model-127: 2799it [00:00, 27413.43it/s]?, ?it/s]
Running Model-127: 100%|██████████| 1/1 [00:00<00:00,  9.42it/s]


Fitting node Ridge-85...


Running Model-127: 2799it [00:00, 28387.66it/s]         

==================NARMA NRMSE==================
Normal training error: 0.8012378809512005
NVAR training error: 0.8189327726285235
Cascading ESN error: 0.7861238499696007
Deep ESN error: 0.6982829047067439
Deep + cascade error: 0.7168850606564878
Deep + cascade + NVAR error: 2.011018874081668
Error cascade error: 0.7118940944934266
Deep error cascade error: 0.7831800231990088


In [38]:
from bayes_opt import BayesianOptimization
import random

def testDefault(**params):
    esn = (NVAR(delay=int(params['delay']), order=int(params['order']), strides=int(params['strides'])) >> Reservoir(units=int(params['units']), sr=params['sr'], rc_connectivity=params['con'])) >> Ridge(output_dim=1, ridge=params['ridge'])
    trainX, trainY, testX, testY = getData()

    esn.fit(trainX, trainY, warmup=1000)

    esn2 = (NVAR(delay=int(params['delay']), order=int(params['order']), strides=int(params['strides'])) >> Reservoir(units=int(params['units']), sr=params['sr'], rc_connectivity=params['con'])) >> Ridge(output_dim=1, ridge=params['ridge'])
    esnPreds = copy.deepcopy(esn).run(trainX)
    esn2.fit(esnPreds, trainY, warmup=1000)

    preds1 = esn.run(testX)
    preds2 = esn2.run(preds1)

    return 1/nrmse(testY, preds2)

pbounds = {'delay': (2, 16), 'order': (1, 10), 'strides': (2, 10), 'units': (50, 200), 'sr': (0.1, 1.2), 'con': (0.001, 0.02), 'ridge': (1e-6, 1e-4)}

optimizer = BayesianOptimization(
    f=testDefault,
    pbounds=pbounds,
    random_state=1,
)

optimizer.maximize(
    init_points=20,
    n_iter=25,
)

|   iter    |  target   |    con    |   delay   |   order   |   ridge   |    sr     |  strides  |   units   |
-------------------------------------------------------------------------------------------------------------


Running Model-1195: 5599it [00:00, 13043.87it/s]?, ?it/s]
Running Model-1195: 100%|██████████| 1/1 [00:00<00:00,  2.31it/s]


Fitting node Ridge-283...


Running Model-1195: 5599it [00:00, 14775.43it/s]         
Running Model-1197: 5599it [00:00, 13395.31it/s]?, ?it/s]
Running Model-1197: 100%|██████████| 1/1 [00:00<00:00,  2.37it/s]


Fitting node Ridge-284...


...

Fitting node Ridge-371...


Running Model-1371: 5599it [00:00, 10695.59it/s]         
Running Model-1373: 5599it [00:00, 8311.41it/s]<?, ?it/s]
Running Model-1373: 100%|██████████| 1/1 [00:00<00:00,  1.47it/s]


Fitting node Ridge-372...


Running Model-1371: 2399it [00:00, 10838.60it/s]         
Running Model-1373: 2399it [00:00, 9795.15it/s]          

| 45        | 5.744     | 0.005717  | 10.59     | 4.57      | 8.864e-05 | 0.7876    | 3.695     | 130.3     |


In [40]:
1/optimizer.max['target']

0.16941265512973996